In [3]:
import pandas as pd
df_label=pd.read_csv("dataset/recurrence.csv")

/tmp/ipykernel_30023/676453913.py:2: DtypeWarning: Columns (5,9,33,34,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_label=pd.read_csv("dataset/recurrence.csv")


In [4]:
df_label.head(3)

,Unnamed: 0,image_path,Patient ID,Patient affiliation,Age at Histological Diagnosis,Weight (lbs),Gender,Ethnicity,Smoking status,Pack Years,...,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
0,0,/kaggle/input/nsclc-radiogenomics-6-1-21-versi...,AMC-001,Stanford,34,Not Collected,Male,Not Recorded In Database,Nonsmoker,NaN,...,04-30-1994,WB NAC P690,GE MEDICAL SYSTEMS,PT,Positron Emission Tomography Image Storage,1.2.840.10008.5.1.4.1.1.128,263,20.38 MB,.\NSCLC Radiogenomics\AMC-001\04-30-1994-NA-PE...,2024-08-26T12:09:15.173
1,1,/kaggle/input/nsclc-radiogenomics-6-1-21-versi...,AMC-001,Stanford,34,Not Collected,Male,Not Recorded In Database,Nonsmoker,NaN,...,04-30-1994,WB NAC P690,GE MEDICAL SYSTEMS,PT,Positron Emission Tomography Image Storage,1.2.840.10008.5.1.4.1.1.128,263,20.38 MB,.\NSCLC Radiogenomics\AMC-001\04-30-1994-NA-PE...,2024-08-26T12:09:15.173
2,2,/kaggle/input/nsclc-radiogenomics-6-1-21-versi...,AMC-001,Stanford,34,Not Collected,Male,Not Recorded In Database,Nonsmoker,NaN,...,04-30-1994,WB NAC P690,GE MEDICAL SYSTEMS,PT,Positron Emission Tomography Image Storage,1.2.840.10008.5.1.4.1.1.128,263,20.38 MB,.\NSCLC Radiogenomics\AMC-001\04-30-1994-NA-PE...,2024-08-26T12:09:15.173


In [5]:
df_label['SOPInstanceUID'].nunique()

285093

In [6]:
import os
import pandas as pd

def gather_image_paths(root_folder):
    image_paths = []
    
    # Walk through the directory
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            # Check if the file is an image (you can add more extensions as needed)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif','.dcm','.tiff')):
                # Create the full path to the image
                full_path = os.path.join(subdir, file)
                image_paths.append(full_path)
                
    return image_paths

In [7]:
root_folder = 'dataset/ROI'
image_paths = gather_image_paths(root_folder)

# Create a DataFrame
df_images = pd.DataFrame(image_paths, columns=['image_path'])

In [8]:
# Create SOPInstanceUID column
df_images['SOPInstanceUID'] = df_images['image_path'].str.split('/').str[-1]  # Extract the final part
df_images['SOPInstanceUID'] = df_images['SOPInstanceUID'].str.replace(r'(_.*|\.tiff)', '', regex=True)  # Remove .tiff and anything behind "_"

In [9]:
# Perform a left join to add the 'Recurrence' column from df_label to df_images using 'SOPInstanceUID' as the key
df = df_images.merge(df_label[['SOPInstanceUID', 'Recurrence']], on='SOPInstanceUID', how='left')

In [10]:
df_selected=df[['image_path','Recurrence','SOPInstanceUID']]

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Step 1: Group by SOPInstanceUID
unique_uids = df_selected['SOPInstanceUID'].unique()

# Step 2: Map SOPInstanceUID to Recurrence (one Recurrence per unique SOPInstanceUID)
uid_to_recurrence = df_selected.drop_duplicates(subset=['SOPInstanceUID'])[['SOPInstanceUID', 'Recurrence']]

# Step 3: First split into training and temp sets based on unique SOPInstanceUID
train_uids, temp_uids = train_test_split(
    unique_uids, 
    test_size=0.4,  # 40% of the unique UIDs go to temp
    stratify=uid_to_recurrence['Recurrence'],  # Ensure stratification using the reduced Recurrence labels
    random_state=42  # For reproducibility
)

# Step 4: Second split temp_uids into validation and test sets
val_uids, test_uids = train_test_split(
    temp_uids,
    test_size=0.5,  # Split temp_uids into two equal parts for validation and test
    stratify=uid_to_recurrence[uid_to_recurrence['SOPInstanceUID'].isin(temp_uids)]['Recurrence'],  # Stratify within temp
    random_state=42  # For reproducibility
)

# Step 5: Create the final dataframes by filtering the original dataframe
train_df = df_selected[df_selected['SOPInstanceUID'].isin(train_uids)]
val_df = df_selected[df_selected['SOPInstanceUID'].isin(val_uids)]
test_df = df_selected[df_selected['SOPInstanceUID'].isin(test_uids)]

In [12]:
train_df.to_csv("dataset/train.csv")
test_df.to_csv("dataset/test.csv")
val_df.to_csv("dataset/val.csv")